In [8]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
import os

# TASK 1: Tải và Tiền xử lý Dữ liệu

In [9]:
def load_conllu(file_path):
    sentences = []
    current_sentence = []
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            
            # Bỏ qua các dòng comment hoặc dòng trống
            if not line or line.startswith('#'):
                if line == '' and current_sentence:
                    sentences.append(current_sentence)
                    current_sentence = []
                continue
            
            # Parse dòng dữ liệu
            fields = line.split('\t')
            if len(fields) >= 4 and fields[0].isdigit():
                word = fields[1]  # Cột 2 (FORM)
                upos_tag = fields[3]  # Cột 4 (UPOS)
                current_sentence.append((word, upos_tag))
        
        # Thêm câu cuối cùng nếu có
        if current_sentence:
            sentences.append(current_sentence)
    
    return sentences


def build_vocabulary(sentences):
    word_counter = Counter()
    tag_counter = Counter()
    
    for sentence in sentences:
        for word, tag in sentence:
            word_counter[word] += 1
            tag_counter[tag] += 1
    
    # Tạo word_to_ix với token đặc biệt <UNK> và <PAD>
    word_to_ix = {'<PAD>': 0, '<UNK>': 1}
    for word in word_counter:
        word_to_ix[word] = len(word_to_ix)
    
    # Tạo tag_to_ix với token <PAD>
    tag_to_ix = {'<PAD>': 0}
    for tag in tag_counter:
        tag_to_ix[tag] = len(tag_to_ix)
    
    return word_to_ix, tag_to_ix

# TASK 2: Tạo PyTorch Dataset và DataLoader

In [10]:
class POSDataset(Dataset):
    
    def __init__(self, sentences, word_to_ix, tag_to_ix):
        self.sentences = sentences
        self.word_to_ix = word_to_ix
        self.tag_to_ix = tag_to_ix
    
    def __len__(self):
        return len(self.sentences)
    
    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        
        # Chuyển từ và nhãn thành indices
        word_indices = [self.word_to_ix.get(word, self.word_to_ix['<UNK>']) 
                        for word, _ in sentence]
        tag_indices = [self.tag_to_ix[tag] for _, tag in sentence]
        
        return torch.tensor(word_indices, dtype=torch.long), \
               torch.tensor(tag_indices, dtype=torch.long)


def collate_fn(batch):
    sentences, tags = zip(*batch)
    
    # Padding sequences
    sentences_padded = pad_sequence(sentences, batch_first=True, padding_value=0)
    tags_padded = pad_sequence(tags, batch_first=True, padding_value=0)
    
    return sentences_padded, tags_padded

# TASK 3: Xây dựng Mô hình RNN

In [11]:
class SimpleRNNForTokenClassification(nn.Module):
    """Mô hình RNN đơn giản cho bài toán POS Tagging."""
    
    def __init__(self, vocab_size, tagset_size, embedding_dim, hidden_dim):
        super(SimpleRNNForTokenClassification, self).__init__()
        
        # Lớp Embedding
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        
        # Lớp RNN
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        
        # Lớp Linear để dự đoán nhãn
        self.fc = nn.Linear(hidden_dim, tagset_size)
    
    def forward(self, sentences):
        # sentences: (batch_size, seq_len)
        
        # Embedding: (batch_size, seq_len, embedding_dim)
        embeds = self.embedding(sentences)
        
        # RNN: rnn_out shape: (batch_size, seq_len, hidden_dim)
        rnn_out, _ = self.rnn(embeds)
        
        # Linear: (batch_size, seq_len, tagset_size)
        tag_space = self.fc(rnn_out)
        
        return tag_space

# TASK 4: Huấn luyện Mô hình

In [12]:
def train_model(model, train_loader, optimizer, criterion, device):
    """Huấn luyện mô hình trên một epoch."""
    model.train()
    total_loss = 0
    
    for sentences, tags in train_loader:
        sentences, tags = sentences.to(device), tags.to(device)
        
        # 1. Xóa gradient cũ
        optimizer.zero_grad()
        
        # 2. Forward pass
        outputs = model(sentences)
        
        # 3. Tính loss
        # Reshape outputs và tags để tính loss
        outputs = outputs.view(-1, outputs.shape[-1])  # (batch_size * seq_len, tagset_size)
        tags = tags.view(-1)  # (batch_size * seq_len)
        
        loss = criterion(outputs, tags)
        
        # 4. Backward pass
        loss.backward()
        
        # 5. Cập nhật trọng số
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(train_loader)

# TASK 5: Đánh giá Mô hình

In [13]:
def evaluate(model, data_loader, device):
    """Đánh giá mô hình trên tập dữ liệu."""
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for sentences, tags in data_loader:
            sentences, tags = sentences.to(device), tags.to(device)
            
            # Forward pass
            outputs = model(sentences)
            
            # Lấy dự đoán
            predictions = torch.argmax(outputs, dim=-1)
            
            # Tính accuracy (chỉ tính trên token không phải padding)
            mask = tags != 0  # Mask cho padding
            correct += ((predictions == tags) & mask).sum().item()
            total += mask.sum().item()
    
    accuracy = correct / total if total > 0 else 0
    return accuracy


def predict_sentence(model, sentence, word_to_ix, ix_to_tag, device):
    """
    Dự đoán POS tags cho một câu mới.
    sentence: chuỗi văn bản, ví dụ: "I love NLP"
    """
    model.eval()
    
    # Tách câu thành các từ
    words = sentence.split()
    
    # Chuyển từ thành indices
    word_indices = [word_to_ix.get(word, word_to_ix['<UNK>']) for word in words]
    sentence_tensor = torch.tensor([word_indices], dtype=torch.long).to(device)
    
    with torch.no_grad():
        outputs = model(sentence_tensor)
        predictions = torch.argmax(outputs, dim=-1)
    
    # Chuyển predictions thành tags
    predicted_tags = [ix_to_tag[idx.item()] for idx in predictions[0]]
    
    return list(zip(words, predicted_tags))

# Main

In [14]:
def main():
    # Thiết lập device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Đường dẫn đến dữ liệu
    data_dir = r'D:\10. ky1nam4\NLP\data\UD_English-EWT'
    train_file = os.path.join(data_dir, 'en_ewt-ud-train.conllu')
    dev_file = os.path.join(data_dir, 'en_ewt-ud-dev.conllu')
    
    # Task 1: Load dữ liệu
    print("\n=== TASK 1: Loading and Preprocessing Data ===")
    train_sentences = load_conllu(train_file)
    dev_sentences = load_conllu(dev_file)
    
    print(f"Number of training sentences: {len(train_sentences)}")
    print(f"Number of dev sentences: {len(dev_sentences)}")
    print(f"Example sentence: {train_sentences[0][:5]}")
    
    # Xây dựng vocabulary
    word_to_ix, tag_to_ix = build_vocabulary(train_sentences)
    ix_to_tag = {v: k for k, v in tag_to_ix.items()}
    
    print(f"\nVocabulary size: {len(word_to_ix)}")
    print(f"Tagset size: {len(tag_to_ix)}")
    print(f"Tags: {list(tag_to_ix.keys())}")
    
    # Task 2: Tạo Dataset và DataLoader
    print("\n=== TASK 2: Creating Dataset and DataLoader ===")
    train_dataset = POSDataset(train_sentences, word_to_ix, tag_to_ix)
    dev_dataset = POSDataset(dev_sentences, word_to_ix, tag_to_ix)
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
    dev_loader = DataLoader(dev_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)
    
    print(f"Number of training batches: {len(train_loader)}")
    print(f"Number of dev batches: {len(dev_loader)}")
    
    # Task 3: Xây dựng mô hình
    print("\n=== TASK 3: Building RNN Model ===")
    EMBEDDING_DIM = 100
    HIDDEN_DIM = 128
    
    model = SimpleRNNForTokenClassification(
        vocab_size=len(word_to_ix),
        tagset_size=len(tag_to_ix),
        embedding_dim=EMBEDDING_DIM,
        hidden_dim=HIDDEN_DIM
    ).to(device)
    
    print(f"Model architecture:\n{model}")
    
    # Task 4 & 5: Huấn luyện và đánh giá
    print("\n=== TASK 4 & 5: Training and Evaluation ===")
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss(ignore_index=0)  # Ignore padding
    
    NUM_EPOCHS = 10
    best_dev_acc = 0
    
    for epoch in range(NUM_EPOCHS):
        # Huấn luyện
        train_loss = train_model(model, train_loader, optimizer, criterion, device)
        
        # Đánh giá
        train_acc = evaluate(model, train_loader, device)
        dev_acc = evaluate(model, dev_loader, device)
        
        print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
        print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"  Dev Acc: {dev_acc:.4f}")
        
        # Lưu mô hình tốt nhất
        if dev_acc > best_dev_acc:
            best_dev_acc = dev_acc
            torch.save(model.state_dict(), 'best_model.pt')
            print(f"  -> New best model saved!")
    
    # Load mô hình tốt nhất và đánh giá
    print("\n=== Final Results ===")
    model.load_state_dict(torch.load('best_model.pt'))
    final_dev_acc = evaluate(model, dev_loader, device)
    print(f"Best Dev Accuracy: {final_dev_acc:.4f}")
    
    # Test dự đoán câu mới
    print("\n=== Predicting New Sentences ===")
    test_sentences = [
        "I love NLP",
        "The cat sits on the mat",
        "She is reading a book"
    ]
    
    for sentence in test_sentences:
        predictions = predict_sentence(model, sentence, word_to_ix, ix_to_tag, device)
        print(f"\nSentence: '{sentence}'")
        print(f"Predictions: {predictions}")


if __name__ == "__main__":
    main()

Using device: cpu

=== TASK 1: Loading and Preprocessing Data ===
Number of training sentences: 12544
Number of dev sentences: 2001
Example sentence: [('Al', 'PROPN'), ('-', 'PUNCT'), ('Zaman', 'PROPN'), (':', 'PUNCT'), ('American', 'ADJ')]

Vocabulary size: 19675
Tagset size: 18
Tags: ['<PAD>', 'PROPN', 'PUNCT', 'ADJ', 'NOUN', 'VERB', 'DET', 'ADP', 'AUX', 'PRON', 'PART', 'SCONJ', 'NUM', 'ADV', 'CCONJ', 'X', 'INTJ', 'SYM']

=== TASK 2: Creating Dataset and DataLoader ===
Number of training batches: 392
Number of dev batches: 63

=== TASK 3: Building RNN Model ===
Model architecture:
SimpleRNNForTokenClassification(
  (embedding): Embedding(19675, 100, padding_idx=0)
  (rnn): RNN(100, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=18, bias=True)
)

=== TASK 4 & 5: Training and Evaluation ===
Epoch 1/10
  Train Loss: 1.1228 | Train Acc: 0.7700
  Dev Acc: 0.7477
  -> New best model saved!
Epoch 2/10
  Train Loss: 0.6224 | Train Acc: 0.8384
  Dev Acc: 0.8033
  -> New b

# BÁO CÁO LAB 5: Xây dựng mô hình RNN cho bài toán POS Tagging

## 1. Mục tiêu
Trong bài thực hành này, chúng ta áp dụng kiến thức về mạng nơ-ron hồi quy (RNN) để xây dựng mô hình dự đoán nhãn Part-of-Speech (POS) cho từng token trong câu. Các bước chính:
- Tiền xử lý dữ liệu từ định dạng CoNLL-U.
- Xây dựng từ điển cho từ và nhãn.
- Tạo Dataset và DataLoader trong PyTorch.
- Xây dựng mô hình RNN đơn giản với các lớp `Embedding`, `RNN`, và `Linear`.
- Huấn luyện và đánh giá mô hình.

## 2. Bộ dữ liệu
- Sử dụng bộ dữ liệu **UD_English-EWT** ở định dạng CoNLL-U.
- Số lượng câu:
  - Train: **12,544 câu**
  - Dev: **2,001 câu**
- Ví dụ một câu sau khi xử lý: [('Al', 'PROPN'), ('-', 'PUNCT'), ('Zaman', 'PROPN'), (':', 'PUNCT'), ('American', 'ADJ')]
- Kích thước từ điển:
- Vocabulary: **19,675 từ**
- Tagset: **18 nhãn**
- Các nhãn: `<PAD>`, PROPN, PUNCT, ADJ, NOUN, VERB, DET, ADP, AUX, PRON, PART, SCONJ, NUM, ADV, CCONJ, X, INTJ, SYM.

## 3. Mô hình
- Kiến trúc: SimpleRNNForTokenClassification(
(embedding): Embedding(19675, 100, padding_idx=0)
(rnn): RNN(100, 128, batch_first=True)
(fc): Linear(in_features=128, out_features=18, bias=True)
)

## 4. Huấn luyện
- Số epoch: **10**
- Optimizer: Adam
- Loss: CrossEntropyLoss (ignore_index cho padding)
- Kết quả huấn luyện:

| Epoch | Train Loss | Train Acc | Dev Acc |
|-------|-----------|-----------|---------|
| 1     | 1.1228    | 0.7700    | 0.7477 |
| 5     | 0.3058    | 0.9208    | 0.8548 |
| 10    | 0.1275    | 0.9684    | 0.8744 |

- **Best Dev Accuracy:** **0.8744**

## 5. Dự đoán câu mới
- **Câu:** "I love NLP"  
**Dự đoán:** `[('I', 'PRON'), ('love', 'VERB'), ('NLP', 'NOUN')]`

- **Câu:** "The cat sits on the mat"  
**Dự đoán:** `[('The', 'DET'), ('cat', 'NOUN'), ('sits', 'NOUN'), ('on', 'ADP'), ('the', 'DET'), ('mat', 'NOUN')]`

- **Câu:** "She is reading a book"  
**Dự đoán:** `[('She', 'PRON'), ('is', 'AUX'), ('reading', 'VERB'), ('a', 'DET'), ('book', 'NOUN')]`